In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hydrant.topology.geom as gm
import subprocess
import os
from   shapely.geometry import Point
import yaml
import shutil
import warnings
import networkx as nx
import re
import copy


warnings.filterwarnings("ignore")

from riverlakenetwork import DataChecker, DataLoader
from riverlakenetwork import ResolvableLakes
from riverlakenetwork import BurnLakes

In [2]:
# Load general configurations from a YAML file
yaml_file = os.path.abspath('../code/HydroLakes_MERITBasins/HydroLakes_MERITBasins_Config.yaml')
with open(yaml_file, 'r') as f:
    GeneralConfig = yaml.safe_load(f)

# get the path to the merit-basins files
path_out             = GeneralConfig['DomainFolder']
TempFolder           = GeneralConfig['TempFolder']
pfafs                = GeneralConfig['pfafs']
lake_file            = GeneralConfig['LakeFile']
IncludeLakes         = GeneralConfig['IncludeLakes']
riv_path             = GeneralConfig['MERITBasinsPaths']['RivPath']
cat_path             = GeneralConfig['MERITBasinsPaths']['CatPath']
cst_path             = GeneralConfig['MERITBasinsPaths']['CstPath']
riv_file_template    = GeneralConfig['MERITBasinsPaths']['RivFileTemplate']
cat_file_template    = GeneralConfig['MERITBasinsPaths']['CatFileTemplate']
cst_file_template    = GeneralConfig['MERITBasinsPaths']['CstFileTemplate']

In [3]:
if IncludeLakes:
    #load hydrolakeDataset
    lakeO = gpd.read_file(lake_file) # read the hydrolake dataset
    # lakeO = lakeO.sample(frac=0.10, random_state=42)
    #lakeO = FixHydroLakesForMerit(lakeO) # correct for hydrolakes for merit
    lakeO['x'] = lakeO.centroid.x # add centroid lon
    lakeO['y'] = lakeO.centroid.y # add centroid lat

# loop over the config files:
for pfaf in ['71']:

    # get the list of pfaf for the subdomain
    subdomain = 'pfaf'+pfaf
    
    # create the folder and domain to include
    path_out_temp = TempFolder+'pfaffff'+pfaf+'/'+subdomain+'/'
    if os.path.isdir(path_out_temp):
        try:
            shutil.rmtree(path_out_temp)  # Remove the entire directory and its contents
        except OSError as e:
            print(f"Error: {path_out_temp} - {e}")
    if not os.path.isdir(path_out_temp):
        os.makedirs(path_out_temp)

    # read in the files
    riv, cat = gm.merit_read_file ([pfaf],
                                   riv_path,
                                   riv_file_template,
                                   cat_path,
                                   cat_file_template,
                                   path_cst = cst_path,
                                   cst_file_template = cst_file_template)
    
    # prepare the ntopo for merit
    riv, cat = gm.prepare_ntopo(riv=riv,\
                                riv_cols={'id':'COMID', 'next_id':'NextDownID'},\
                                cat=cat,\
                                cat_cols={'id':'COMID', 'hillslope':'hillslope', 'area' :'unitarea'},\
                                network = 'merit')

In [4]:
import riverlakenetwork as rlt

# use DataLoader to
# Example config dictionary
config = {
    "riv": riv,
    "riv_dict": {
        "COMID": {"col":"COMID"},
        "NextDownCOMID": {"col":"NextDownID"},
        "length": {"col":"lengthkm"},
        "uparea": {"col":"uparea","unit":"km2"}
    },

    "cat": cat,
    "cat_dict": {
        "COMID": {"col":"COMID"},
        "unitarea": {"col":"unitarea","unit":"km2"},
    },

    "lake": lakeO,
    "lake_dict": {
        "LakeCOMID": {"col":"Hylak_id"},
        "unitarea": {"col":"Lake_area","unit":"km2"}
    }
}

# Create DataLoader instance
RLT = rlt.DataLoader(config)

# Access the layers
riv = RLT.riv
cat = RLT.cat
lake = RLT.lake


riv: Loaded
riv_dict: {'COMID': {'col': 'COMID'}, 'NextDownCOMID': {'col': 'NextDownID'}, 'length': {'col': 'lengthkm'}, 'uparea': {'col': 'uparea', 'unit': 'km2'}}
cat: Loaded
cat_dict: {'COMID': {'col': 'COMID'}, 'unitarea': {'col': 'unitarea', 'unit': 'km2'}}
lake: Loaded
lake_dict: {'LakeCOMID': {'col': 'Hylak_id'}, 'unitarea': {'col': 'Lake_area', 'unit': 'km2'}}


In [5]:
lake


,Hylak_id,Lake_name,Country,Continent,Poly_src,Lake_type,Grand_id,Lake_area,Shore_len,Shore_dev,...,Dis_avg,Res_time,Elevation,Slope_100,Wshd_area,Pour_long,Pour_lat,geometry,x,y
0,1,Caspian Sea,Russia,Europe,SWBD,1,0,377001.91,15829.37,7.27,...,8110.642,107883.0,-29,-1.00,1404108.0,47.717708,45.591934,"POLYGON ((49.96181 37.43847, 49.96457 37.44022...",50.735479,41.713338
1,2,Great Bear,Canada,North America,CanVec,1,0,30450.64,5331.72,8.62,...,535.187,47577.7,145,-1.00,147665.4,-123.505546,65.138384,"POLYGON ((-119.78782 67.03574, -119.78637 67.0...",-120.967234,65.999359
2,3,Great Slave,Canada,North America,CanVec,1,0,26734.29,9814.16,16.93,...,4350.692,4203.2,148,-1.00,995312.3,-117.617115,61.311727,"POLYGON ((-109.93976 62.95851, -109.93831 62.9...",-113.742931,61.784589
3,4,Winnipeg,Canada,North America,CanVec,3,709,23923.04,4018.32,7.33,...,2244.727,1464.3,215,-1.00,919611.5,-97.863542,53.696359,"POLYGON ((-98.80636 53.88021, -98.80578 53.880...",-97.806211,52.532676
4,5,Superior,United States of America,North America,SWBD,1,0,81843.92,5248.36,5.18,...,2869.953,48410.3,179,-1.00,209219.5,-84.460547,46.468593,"POLYGON ((-90.7225 46.6574, -90.72458 46.65783...",-87.756530,47.526524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427683,1427684,None,New Zealand,Oceania,SWBD,1,0,0.29,2.30,1.20,...,0.125,139.6,1,4.83,4.4,169.147152,-52.592307,"POLYGON ((169.14763 -52.59291, 169.1475 -52.59...",169.147186,-52.588635
1427684,1427685,None,Australia,Oceania,SWBD,1,0,0.10,1.66,1.52,...,-9999.000,-9999.0,1,1.04,-9999.0,73.305108,-52.973622,"POLYGON ((73.30605 -52.97318, 73.3054 -52.9746...",73.303794,-52.972125
1427685,1427686,None,Australia,Oceania,SWBD,1,0,0.24,1.92,1.10,...,0.013,792.4,205,2.63,0.2,158.892278,-54.529970,"POLYGON ((158.89124 -54.53117, 158.88696 -54.5...",158.888990,-54.528658
1427686,1427687,None,Australia,Oceania,SWBD,1,0,0.34,2.95,1.44,...,0.020,1263.9,163,7.58,0.4,158.889583,-54.594300,"POLYGON ((158.88858 -54.59767, 158.88833 -54.5...",158.890587,-54.593975


In [6]:
checker = DataChecker(loaded_data=RLT)
checker

Subbasin and lake area units are consistent: km2
riv CRS: EPSG:4326
cat CRS: EPSG:4326
lake CRS: EPSG:4326


In [7]:
checker.riv

,COMID,NextDownCOMID,length,uparea,geometry,unitarea
0,71000001,71000065,7.020492,391.901537,"LINESTRING (-98.06167 60, -98.0625 59.99917, -...",39.620826
1,71000002,71000065,24.171249,701.754912,"LINESTRING (-98.06167 60, -98.06083 60.00083, ...",99.276774
2,71000003,71000002,14.760464,480.760660,"LINESTRING (-98.1725 60.13, -98.17333 60.12917...",63.957629
3,71000004,71000003,6.240078,388.938962,"LINESTRING (-98.335 60.07, -98.33583 60.07083,...",16.746171
4,71000005,71000004,4.462477,328.557527,"LINESTRING (-98.39917 60.09667, -98.4 60.09583...",5.993164
...,...,...,...,...,...,...
50682,71050683,-9999,0.001000,0.060656,None,0.060656
50683,71050684,-9999,0.001000,0.008723,None,0.008723
50684,71050685,-9999,0.001000,0.004362,None,0.004362
50685,71050686,-9999,0.001000,10.810153,None,10.810153


In [8]:
checker.cat

,COMID,unitarea,geometry
0,71000001,39.620826,"POLYGON ((-98.14542 60.06375, -98.13792 60.063..."
1,71000002,99.276774,"MULTIPOLYGON (((-98.11958 60.19292, -98.11458 ..."
2,71000003,63.957629,"MULTIPOLYGON (((-98.33375 60.03542, -98.33292 ..."
3,71000004,16.746171,"MULTIPOLYGON (((-98.40125 60.05208, -98.40042 ..."
4,71000005,5.993164,"POLYGON ((-98.43208 60.10708, -98.43125 60.107..."
...,...,...,...
50682,71050683,0.060656,"POLYGON ((-94.80792 59.88292, -94.80458 59.882..."
50683,71050684,0.008723,"POLYGON ((-94.80875 59.65792, -94.80708 59.657..."
50684,71050685,0.004362,"POLYGON ((-94.81042 59.65792, -94.80958 59.657..."
50685,71050686,10.810153,"POLYGON ((-108.54708 52.72625, -108.54625 52.7..."


In [9]:
checker.lake

,LakeCOMID,unitarea,geometry
0,1,377001.91,"POLYGON ((49.96181 37.43847, 49.96457 37.44022..."
1,2,30450.64,"POLYGON ((-119.78782 67.03574, -119.78637 67.0..."
2,3,26734.29,"POLYGON ((-109.93976 62.95851, -109.93831 62.9..."
3,4,23923.04,"POLYGON ((-98.80636 53.88021, -98.80578 53.880..."
4,5,81843.92,"POLYGON ((-90.7225 46.6574, -90.72458 46.65783..."
...,...,...,...
1427683,1427684,0.29,"POLYGON ((169.14763 -52.59291, 169.1475 -52.59..."
1427684,1427685,0.10,"POLYGON ((73.30605 -52.97318, 73.3054 -52.9746..."
1427685,1427686,0.24,"POLYGON ((158.89124 -54.53117, 158.88696 -54.5..."
1427686,1427687,0.34,"POLYGON ((158.88858 -54.59767, 158.88833 -54.5..."


In [10]:
resolvable = ResolvableLakes(cat=checker.cat, lake=checker.lake, riv=checker.riv, margin=2.0)



Index(['LakeCOMID', 'unitarea_left', 'geometry', 'x', 'y', 'index_right',
       'COMID', 'unitarea_right'],
      dtype='object')
==== Number of lakes after subsetting: 140586 ====
==== Number of lakes after removing intersection with only one lake: 5202 ====
==== Number of lakes after removing lakes that do not touch starting or ending points of river segments: 4069 ====
==== Number of lakes after removing lakes that do touch only one starting or ending points of river segments: 4007 ====
Number of lakes in the intersection: 4007
Number of river segments intersecting more than one lake: 2292
Number of lakes after removing single-segment lakes: 4007
==== Number of lakes after removing lakes that do intersect with only one river segment: 4007 ====
Removed 0 lakes where max uparea repeats across multiple COMIDs.
==== Number of lakes after removing lakes that do have exactly the same uparea for their maximume uparea for various: 4007 ====
Removed 7 unresolved lakes (graph-based check).
=

In [11]:
resolvable.lake_resolvable.to_file("/Users/shg096/Desktop/LakeRiverOut/Tempdomain/pfaffff71/test.gpkg")

In [12]:
corrected = BurnLakes(cat=checker.cat, lake=resolvable.lake_resolvable, riv=checker.riv)

corrected.cat_corrected.to_file("/Users/shg096/Desktop/LakeRiverOut/Tempdomain/pfaffff71/cat1.gpkg")
corrected.riv_corrected.to_file("/Users/shg096/Desktop/LakeRiverOut/Tempdomain/pfaffff71/riv1.gpkg")

In [13]:
def is_exhoreic_flag(val):
    """
    Robustly check if lake is exhoreic.
    Accepts int, float, string representations of 0/1.
    Returns True only if val represents 1; False otherwise.
    """
    if val is None:
        return False
    if pd.isna(val):
        return False
    try:
        # convert to float first, then check if == 1
        return float(val) == 1
    except (ValueError, TypeError):
        return False

print(is_exhoreic_flag('f'))

False


In [14]:
corrected.riv_corrected

,COMID,NextDownCOMID,length,uparea,geometry,unitarea,LakeCOMID,exorheic,endorheic,islake,inflow,outflow,inoutflow
0,71000001,408.0,0.000000,391.901537,None,25.264515,NaN,NaN,NaN,NaN,1,0,0
1,71000002,408.0,0.000019,701.754912,"LINESTRING (-98.06166 60.00001, -98.06166 60.0...",71.837909,NaN,NaN,NaN,NaN,1,0,0
2,71000003,408.0,9.646425,480.760660,"MULTILINESTRING ((-98.17611 60.12917, -98.1766...",61.706085,NaN,NaN,NaN,NaN,1,1,1
3,71000004,60974.0,4.533462,388.938962,"LINESTRING (-98.34031 60.07281, -98.34083 60.0...",15.942479,NaN,NaN,NaN,NaN,1,1,1
4,71000005,643168.0,1.571774,328.557527,"MULTILINESTRING ((-98.40463 60.095, -98.40487 ...",5.518182,NaN,NaN,NaN,NaN,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54682,71054683,71004303.0,NaN,NaN,"POLYGON ((-95.90416 56.29273, -95.9041 56.2923...",280.540000,479.0,1.0,0.0,1.0,0,0,0
54683,71054684,71004300.0,NaN,NaN,"POLYGON ((-95.33801 56.36026, -95.33453 56.360...",29.980000,5761.0,1.0,0.0,1.0,0,0,0
54684,71054685,71004297.0,NaN,NaN,"POLYGON ((-95.32913 56.53669, -95.32805 56.536...",316.850000,477.0,1.0,0.0,1.0,0,0,0
54685,71054686,71004294.0,NaN,NaN,"POLYGON ((-94.37527 56.40751, -94.37193 56.404...",29.610000,5737.0,1.0,0.0,1.0,0,0,0
